In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
db_url=os.getenv("DATABASE_URL")

In [3]:
load_dotenv()
con =os.getenv("DATABASE_URL")

In [4]:
engine = create_engine(con)

In [5]:
query= '''
select * from
 "Retail_data" rd ;
 '''  

In [6]:
retail_df= pd.read_sql(query,con=db_url)

In [7]:
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Remove invoice cancellations (InvoiceNo beginning with "C").

In [8]:
clean_df = retail_df[~retail_df['InvoiceNo'].astype(str).str.startswith('C')]


Drop rows where CustomerID is missing.

In [9]:
retail_df = retail_df.dropna(subset=['CustomerID'])


Remove duplicate records.

In [10]:
retail_df = retail_df.drop_duplicates()


In [11]:
retail_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Create new fields:
Revenue = Quantity × UnitPrice

In [12]:
retail_df['Revenue'] = retail_df['Quantity'] * retail_df['UnitPrice']


Month, Year, Day extracted from InvoiceDate

In [13]:
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'])


In [14]:
retail_df['Year'] = retail_df['InvoiceDate'].dt.year


In [15]:
retail_df['Month'] = retail_df['InvoiceDate'].dt.month


In [16]:
retail_df['Day'] = retail_df['InvoiceDate'].dt.day


 Time Series (2011 Revenue by Month)

In [17]:
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'])

df_2011 = retail_df[retail_df['InvoiceDate'].dt.year == 2011]


In [18]:
df_2011['Revenue'] = df_2011['Quantity'] * df_2011['UnitPrice']


C:\Users\USER\AppData\Local\Temp\ipykernel_12844\3480320222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2011['Revenue'] = df_2011['Quantity'] * df_2011['UnitPrice']


In [19]:
monthly_revenue_2011 = (
    df_2011
    .groupby(df_2011['InvoiceDate'].dt.month)['Revenue']
    .sum()
    .reset_index(name='Total_Revenue')
)


In [20]:
print(monthly_revenue_2011)


    InvoiceDate  Total_Revenue
0             1     473731.900
1             2     435534.070
2             3     578576.210
3             4     425222.671
4             5     647011.670
5             6     606862.520
6             7     573112.321
7             8     615078.090
8             9     929356.232
9            10     973306.380
10           11    1126815.070
11           12     341539.430


In [21]:
monthly_revenue_2011['MoM_Change_%'] = (
    monthly_revenue_2011['Total_Revenue'].pct_change() * 100
)


In [22]:
monthly_revenue_2011['MoM_Change_Value'] = (
    monthly_revenue_2011['Total_Revenue'].diff()
)


Country Performance (Excluding United Kingdom)

In [23]:
df_non_uk = retail_df[retail_df['Country'] != 'United Kingdom']


In [24]:
df_non_uk['Revenue'] = df_non_uk['Quantity'] * df_non_uk['UnitPrice']


C:\Users\USER\AppData\Local\Temp\ipykernel_12844\492846182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_uk['Revenue'] = df_non_uk['Quantity'] * df_non_uk['UnitPrice']


In [25]:
country_performance = (
    df_non_uk
    .groupby('Country')
    .agg(
        Total_Revenue=('Revenue', 'sum'),
        Total_Quantity=('Quantity', 'sum')
    )
    .reset_index()
)


In [26]:
top_10_countries = country_performance.sort_values(
    by='Total_Revenue', ascending=False
).head(10)


In [27]:
print(top_10_countries)


        Country  Total_Revenue  Total_Quantity
23  Netherlands      284661.54          200128
10         EIRE      250001.78          136187
14      Germany      221509.47          117341
13       France      196626.05          109806
0     Australia      137009.77           83643
32  Switzerland       55739.40           29778
30        Spain       54756.03           26817
3       Belgium       40910.96           23152
31       Sweden       36585.41           35632
19        Japan       35340.62           25218


 Top Customers by Revenue

In [28]:
retail_df['Revenue'] = retail_df['Quantity'] * retail_df['UnitPrice']


In [29]:
customer_revenue = (
    retail_df
    .groupby('CustomerID')
    .agg(
        Total_Revenue=('Revenue', 'sum'),
        Total_Quantity=('Quantity', 'sum')
    )
    .reset_index()
)


In [30]:
top_10_customers = customer_revenue.sort_values(
    by='Total_Revenue', ascending=False
).head(10)


In [31]:
print(top_10_customers)


      CustomerID  Total_Revenue  Total_Quantity
1703     14646.0      279489.02          196719
4233     18102.0      256438.49           64122
3758     17450.0      187322.17           69009
1895     14911.0      132458.73           77155
55       12415.0      123725.45           77242
1345     14156.0      113214.59           56908
3801     17511.0       88125.38           63012
3202     16684.0       65892.08           49390
1005     13694.0       62690.54           61899
2192     15311.0       59284.19           37673


Global Product Demand

In [34]:
df_global = retail_df[retail_df['Country'] != 'United Kingdom']


quantity sold by country

In [36]:

country_demand = (
    retail_df
    .groupby('Country')
    .agg(Total_Quantity=('Quantity', 'sum'))
    .reset_index()
)
country_demand = country_demand.sort_values(by='Total_Quantity', ascending=False)


print(country_demand)



                 Country  Total_Quantity
35        United Kingdom         3994870
23           Netherlands          200128
10                  EIRE          136187
14               Germany          117341
13                France          109806
0              Australia           83643
31                Sweden           35632
32           Switzerland           29778
30                 Spain           26817
19                 Japan           25218
3                Belgium           23152
24                Norway           19247
26              Portugal           16017
12               Finland           10666
6        Channel Islands            9473
9                Denmark            8188
18                 Italy            7999
7                 Cyprus            6296
29             Singapore            5234
1                Austria            4827
17                Israel            3987
25                Poland            3653
5                 Canada            2763
16              

In [37]:
country_demand = country_demand.sort_values(
    by='Total_Quantity',
    ascending=False
)


In [38]:
top_demand_markets = country_demand.head(10)


In [39]:
print(top_demand_markets)


           Country  Total_Quantity
35  United Kingdom         3994870
23     Netherlands          200128
10            EIRE          136187
14         Germany          117341
13          France          109806
0        Australia           83643
31          Sweden           35632
32     Switzerland           29778
30           Spain           26817
19           Japan           25218
